<a href="https://colab.research.google.com/github/nandyc/rag_basic/blob/main/rag_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# github_email = userdata.get('GITHUB_EMAIL')
# github_username = userdata.get('GITHUB_USERNAME')
# !git config --global user.email {github_email}
# !git config --global user.name {github_username}

In [2]:
!git clone https://github.com/nandyc/rag_basic.git

Cloning into 'rag_basic'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 30 (delta 8), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (30/30), 26.07 KiB | 6.52 MiB/s, done.
Resolving deltas: 100% (8/8), done.


In [3]:
%cd rag_basic

/content/rag_basic


In [4]:
# !git status
# !echo "# Some dummy text" >> new.md
# !git add .
# !git commit -m "test"
# !git push origin main


In [5]:
%%capture
# !pip install python-dotenv
!pip install openai
!pip install chromadb
!pip install tiktoken

In [6]:
import os
import collections
# from dotenv import load_dotenv
from openai import OpenAI
import chromadb
from chromadb.utils import embedding_functions
from google.colab import userdata

In [7]:
# load_dotenv()
# openai_key =os.getenv("OPENAI_API_KEY")

In [8]:
openai_key =userdata.get('OPENAI_APIKEY')

In [9]:
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key=openai_key, model_name="text-embedding-3-small"
)

In [10]:
# Initialize the Chroma client with persistence
chroma_client = chromadb.PersistentClient(path="chroma_persistent_storage")
collection_name = "document_qa_collection"
collection = chroma_client.get_or_create_collection(
    name=collection_name, embedding_function=openai_ef
)

In [11]:
client = OpenAI(api_key=openai_key)

In [12]:
# resp = client.chat.completions.create(
#     model="gpt-3.5-turbo",
#     messages=[
#         {"role": "system", "content": "You are a helpful assistant."},
#         {
#             "role": "user",
#             "content": "What is human life expectancy in the United States?",
#         },
#     ],
# )
# print(resp.choices[0].message.content)
# print (resp)

In [13]:
# Function to load documents from a directory
def load_documents_from_directory(directory_path):
    print("==== Loading documents from directory ====")
    documents = []
    for filename in os.listdir(directory_path):
        if filename.endswith(".txt"):
            with open(
                os.path.join(directory_path, filename), "r", encoding="utf-8"
            ) as file:
                documents.append({"id": filename, "text": file.read()})
    print(type(documents[0]))
    # list_of_filenames = collections.Counter([i['id'] for i in documents]).keys()
    list_of_filenames = [item['id'] for item in documents]
    print(list_of_filenames)
    return documents

In [14]:
# Function to split text into chunks
def split_text(text, chunk_size=1000, chunk_overlap=20):
  chunks = []
  start = 0
  while start < len(text):
    end = start + chunk_size
    chunks.append(text[start:end])
    start = end - chunk_overlap
  print(chunks)
  return chunks

In [15]:
    # Load documents from the directory
directory_path = "./news_articles"
documents = load_documents_from_directory(directory_path)

==== Loading documents from directory ====
<class 'dict'>
['05-03-ai-replace-tv-writers-strike.txt', '05-03-ai-powered-supply-chain-startup-pando-lands-30m-investment.txt']


In [16]:
print(f"Loaded {len(documents)} documents")

Loaded 2 documents


In [17]:
# Split documents into chunks
chunked_documents = []
for doc in documents:
    chunks = split_text(doc["text"])
    print("==== Splitting docs into chunks ====")
    for i, chunk in enumerate(chunks):
        chunked_documents.append({"id": f"{doc['id']}_chunk{i+1}", "text": chunk})

print(f"Split documents into {len(chunked_documents)} chunks")
# print(chunked_documents)

['In the must-watch final season of “Succession,” Kendall Roy enters a conference room with his siblings. As the scene opens, he takes a seat and declares: “Who will be the successor? Me.”\n\nOf course, that scene didn’t appear on HBO’s hit show, but it’s a good illustration of generative AI’s level of sophistication compared to the real thing. Yet as the Writers Guild of America goes on strike in pursuit of livable working conditions and better streaming residuals, the networks won’t budge on writers’ demands to regulate the use of AI in writers’ rooms.\n\n“Our proposal is that we not be required to adapt something that’s output by AI, and that the output of an AI not be considered writers’ work,” comedy writer Adam Conover told TechCrunch. “That doesn’t entirely exclude that technology from the production process, but it does mean that our working conditions wouldn’t be undermined by AI.”\n\nBut the Alliance of Motion Picture and Television Producers (AMPTP) refused to engage with th

In [18]:
# Function to generate embeddings using OpenAI API
def get_openai_embedding(text):
    response = client.embeddings.create(input=text, model="text-embedding-3-small")
    embedding = response.data[0].embedding
    print("==== Generating embeddings... ====")
    return embedding

In [19]:
# Generate embeddings for the document chunks
for doc in chunked_documents:
    print("==== Generating embeddings... ====")
    doc["embedding"] = get_openai_embedding(doc["text"])

# print(doc["embedding"])
print(chunked_documents)

==== Generating embeddings... ====
==== Generating embeddings... ====
==== Generating embeddings... ====
==== Generating embeddings... ====
==== Generating embeddings... ====
==== Generating embeddings... ====
==== Generating embeddings... ====
==== Generating embeddings... ====
==== Generating embeddings... ====
==== Generating embeddings... ====
==== Generating embeddings... ====
==== Generating embeddings... ====
==== Generating embeddings... ====
==== Generating embeddings... ====
==== Generating embeddings... ====
==== Generating embeddings... ====
==== Generating embeddings... ====
==== Generating embeddings... ====
==== Generating embeddings... ====
==== Generating embeddings... ====
==== Generating embeddings... ====
==== Generating embeddings... ====
[{'id': '05-03-ai-replace-tv-writers-strike.txt_chunk1', 'text': 'In the must-watch final season of “Succession,” Kendall Roy enters a conference room with his siblings. As the scene opens, he takes a seat and declares: “Who will 

In [20]:
print(chunked_documents)

[{'id': '05-03-ai-replace-tv-writers-strike.txt_chunk1', 'text': 'In the must-watch final season of “Succession,” Kendall Roy enters a conference room with his siblings. As the scene opens, he takes a seat and declares: “Who will be the successor? Me.”\n\nOf course, that scene didn’t appear on HBO’s hit show, but it’s a good illustration of generative AI’s level of sophistication compared to the real thing. Yet as the Writers Guild of America goes on strike in pursuit of livable working conditions and better streaming residuals, the networks won’t budge on writers’ demands to regulate the use of AI in writers’ rooms.\n\n“Our proposal is that we not be required to adapt something that’s output by AI, and that the output of an AI not be considered writers’ work,” comedy writer Adam Conover told TechCrunch. “That doesn’t entirely exclude that technology from the production process, but it does mean that our working conditions wouldn’t be undermined by AI.”\n\nBut the Alliance of Motion Pi

In [21]:
# Upsert documents with embeddings into Chroma
for doc in chunked_documents:
    print("==== Inserting chunks into db;;; ====")
    collection.upsert(
        ids=[doc["id"]], documents=[doc["text"]], embeddings=[doc["embedding"]]
    )

==== Inserting chunks into db;;; ====
==== Inserting chunks into db;;; ====
==== Inserting chunks into db;;; ====
==== Inserting chunks into db;;; ====
==== Inserting chunks into db;;; ====
==== Inserting chunks into db;;; ====
==== Inserting chunks into db;;; ====
==== Inserting chunks into db;;; ====
==== Inserting chunks into db;;; ====
==== Inserting chunks into db;;; ====
==== Inserting chunks into db;;; ====


In [22]:
# Function to query documents
def query_documents(question, n_results=2):
    # query_embedding = get_openai_embedding(question)
    results = collection.query(query_texts=question, n_results=n_results)

    # Extract the relevant chunks
    relevant_chunks = [doc for sublist in results["documents"] for doc in sublist]
    print("==== Returning relevant chunks ====")
    return relevant_chunks
    # for idx, document in enumerate(results["documents"][0]):
    #     doc_id = results["ids"][0][idx]
    #     distance = results["distances"][0][idx]
    #     print(f"Found document chunk: {document} (ID: {doc_id}, Distance: {distance})")

In [28]:
# Function to generate a response from OpenAI
def generate_response(question, relevant_chunks):
    context = "\n\n".join(relevant_chunks)

    print(f"question type={type(question)}")
    print(f"context type={type(context)}")

    prompt = (
        "You are an assistant for question-answering tasks. Use the following pieces of "
        "retrieved context to answer the question. If you don't know the answer, say that you "
        "don't know. Use three sentences maximum and keep the answer concise."
        "\n\nContext:\n" + context + "\n\nQuestion:\n" + question
    )

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": prompt,
            },
            {
                "role": "user",
                "content": question,
            },
        ],
    )

    answer = response.choices[0].message
    return answer

In [30]:
# Example query
question = "tell me about AI replacing TV writers strike."
# Example query and response generation
# question = "tell me about databricks"
relevant_chunks = query_documents(question)
print(relevant_chunks)
answer = generate_response(question, relevant_chunks)

print(answer)

==== Returning relevant chunks ====
['In the must-watch final season of “Succession,” Kendall Roy enters a conference room with his siblings. As the scene opens, he takes a seat and declares: “Who will be the successor? Me.”\n\nOf course, that scene didn’t appear on HBO’s hit show, but it’s a good illustration of generative AI’s level of sophistication compared to the real thing. Yet as the Writers Guild of America goes on strike in pursuit of livable working conditions and better streaming residuals, the networks won’t budge on writers’ demands to regulate the use of AI in writers’ rooms.\n\n“Our proposal is that we not be required to adapt something that’s output by AI, and that the output of an AI not be considered writers’ work,” comedy writer Adam Conover told TechCrunch. “That doesn’t entirely exclude that technology from the production process, but it does mean that our working conditions wouldn’t be undermined by AI.”\n\nBut the Alliance of Motion Picture and Television Produce